In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import pickle
import pandas as pd

In [2]:
train_set=pd.read_pickle('dataset_next_state/integrators_next_state_train.pickle')
test_set=pd.read_pickle('dataset_next_state/integrators_next_state_test.pickle')

In [3]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.h2o = nn.Linear(hidden_size, output_size)
        self.fc = nn.Linear(output_size, output_size)

    def forward(self, input, hidden):
        #print(f"input is {input}")
        #print(f"hidden is {hidden}")
        combined = torch.cat((input, hidden))
        #print(f"combined is {combined}")
        hidden = self.i2h(combined)
        #print(f"now hidden is {hidden}")
        output = self.h2o(hidden)
        output = self.fc(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(self.hidden_size)

In [4]:
n_iters = 1000
print_every = 1
learning_rate = 0.00005 # If you set this too high, it might explode. If too low, it might not learn
loss_function = nn.MSELoss()
n_hidden = 10
dimension=train_set[0][0].size(0)
rnn = RNN(dimension,n_hidden,dimension)
optimizer=optim.SGD(rnn.parameters(),learning_rate)

In [ ]:
losses=[]
count=0
for iter in range(1, n_iters + 1):
    for trace in train_set:
        hidden=rnn.initHidden()
        for i in range(len(trace)-1):
            #print(hidden)
            output, hidden=rnn(trace[i],hidden)
            hidden=hidden.clone().detach()
            loss=loss_function(output,trace[i+1])
            
            optimizer.zero_grad()
            loss.backward()
            
            optimizer.step()

            losses.append(loss)
            if count % print_every == 0:
                print(loss)
            count+=1

In [ ]:
plt.plot(range(len(losses)),losses)


In [ ]:
def evaluate(nn,trace):
    losses=[]
    hidden = rnn.initHidden()
    for i in range(len(trace)-1):
        output,hidden=nn(trace[i],hidden)
        losses.append(loss_function(output,trace[i+1]))
    

In [ ]:
for trace in test_set:
    losses=[]
    losses.append(*evaluate(rnn,trace))
print(np.mean(losses))
plt.hist(losses)